In [ ]:
import vizdoom as vzd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import itertools as it
import skimage.transform

import os
from time import sleep, time
from collections import deque
from tqdm import trange
import matplotlib.pyplot as plt

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, available_actions_count):
        super(NeuralNet, self).__init__()
        self.convolutional_layers = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3, stride=2, bias=False),
            nn.ReLU(),
            nn.Conv2d(8, 8, kernel_size=3, stride=2, bias=False),
            nn.ReLU(),
            nn.Conv2d(8, 8, kernel_size=3, stride=1, bias=False),
            nn.ReLU(),
            nn.Conv2d(8, 16, kernel_size=3, stride=1, bias=False),
            nn.ReLU()
        )
        self.full_connected_layers = nn.Sequential(
            nn.Linear(192, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
        )

        self.output_layer = nn.Linear(64, available_actions_count)

    def forward(self, x):
        x = self.convolutional_layers(x)
        x = x.view(-1, 192)
        x = self.full_connected_layers(x)
        x = self.output_layer(x)
        return x


In [ ]:
class MCAgent:
    def __init__(self, action_size, params):
        self.action_size = action_size
        self.batch_size = params['batch_size']
        self.discount = params['discount_factor']
        self.learning_rate = params['learning_rate']
        self.epsilon = params['epsilon']
        self.epsilon_decay = params['epsilon_decay']
        self.epsilon_min = params['epsilon_min']
        self.loss_fn = nn.MSELoss()
        self.device = params['device']
        self.episode_with_rewards = deque(maxlen=params['replay_buffer_size'])
        self.dictionary = dict()

        self.behaviour_policy_net = NeuralNet(action_size).to(self.device)

        self.optimizer = optim.SGD(self.behaviour_policy_net.parameters(), lr=self.learning_rate)

    def get_action(self, state):
        if np.random.random() < self.epsilon:
            return random.choice(range(self.action_size))
        else:
            state = self.array_to_tensor(state)
            q_values = self.behaviour_policy_net(state)
            action = q_values.max(dim=1)[1].item()
            return action

    def array_to_tensor(self, arr):
        arr = np.expand_dims(arr, axis=0)
        arr = torch.from_numpy(arr).float().to(self.device)
        return arr

    def update_behaviour_policy_net(self, replay_buffer):
        batch = replay_buffer.copy()
        episode = replay_buffer.copy()
        batch = np.array(batch, dtype=object)

        states = np.stack(batch[:, 0]).astype(float)
        actions = batch[:, 1].astype(int)
        rewards = batch[:, 2].astype(float)
        dones = batch[:, 3].astype(bool)
        not_dones = ~dones
        returns = np.zeros((dones.shape[0]))
        
        g = 0
        for i in reversed(range(len(episode))):
            count = self.dictionary.get(states[i].tobytes())
            if count != None:
                self.dictionary.update({states[i].tobytes(): count + 1})
            else:
                self.dictionary[states[i].tobytes()] = 1
            g = g * self.discount + rewards[i]
            returns[i] = g / self.dictionary.get(states[i].tobytes())

        for i in range(len(episode)):
            self.episode_with_rewards.append((states[i], actions[i], returns[i]))

        sample = random.sample(self.episode_with_rewards, self.batch_size)    
        sample = np.array(sample, dtype=object)

        sample_actions = sample[:, 1].astype(int)
        sample_returns = sample[:, 2].astype(int)
        sample_states = np.stack(sample[:, 0]).astype(float)

        row_idx = np.arange(self.batch_size) 

        q_targets = sample_returns.copy()
        q_targets = torch.from_numpy(q_targets).float().to(self.device)

        idx = row_idx, sample_actions
        sample_states = torch.from_numpy(sample_states).float().to(self.device)
        action_values = self.behaviour_policy_net.forward(sample_states)[idx].float().to(self.device)

        self.optimizer.zero_grad()
        error = self.loss_fn(q_targets, action_values)
        self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        else:
            self.epsilon = self.epsilon_min
        return error.item()

In [ ]:
def preprocess(img, resolution):
    img = skimage.transform.resize(img, resolution)
    img = img.astype(np.float32)
    img = np.expand_dims(img, axis=0)
    return img


In [ ]:
                
def visualize(game, agent, params):
    for _ in range(params['episodes_to_watch']):
        game.new_episode()
        while not game.is_episode_finished():
            state = preprocess(game.get_state().screen_buffer, params['resolution'])
            best_action_index = agent.get_action(state)

            game.set_action(params['actions'][best_action_index])
            for _ in range(params['frame_repeat']):
                game.advance_action()

        sleep(1.0)
        score = game.get_total_reward()
        print("Total score: ", score)

In [ ]:
def train_mc_agent(game, train_parameters):
    actions = train_parameters['actions']

    # Initialize our agent with the set parameters
    agent = MCAgent(len(actions), train_parameters)

    start_time = time()

    time_steps = train_parameters['total_time_steps']
    replay_buffer = deque(maxlen=train_parameters['replay_buffer_size'])

    game.new_episode()
    train_scores = []
    train_loss = []
    global_step = 0

    for _ in trange(time_steps):
        state = preprocess(game.get_state().screen_buffer, train_parameters['resolution'])
        action = agent.get_action(state)
        reward = game.make_action(actions[action], train_parameters['frame_repeat'])
        done = game.is_episode_finished()

        replay_buffer.append((state, action, reward, done))

        if global_step > agent.batch_size and done:
            train_loss.append(agent.update_behaviour_policy_net(replay_buffer))
            replay_buffer.clear()

        if done:
            train_scores.append(game.get_total_reward())
            game.new_episode()

        global_step += 1
    visualize(game, agent, train_parameters)
    train_scores = np.array(train_scores)
    print("Results: mean: %.1f +/- %.1f," % (train_scores.mean(), train_scores.std()),
            "min: %.1f," % train_scores.min(), "max: %.1f," % train_scores.max())

    game.close()
    return agent, game, train_scores, train_loss


In [ ]:
def plot_curves(arr_list, legend_list, color_list, ylabel, fig_title):
    """
    Args:
        arr_list (list): list of results arrays to plot
        legend_list (list): list of legends corresponding to each result array
        color_list (list): list of color corresponding to each result array
        ylabel (string): label of the Y axis

        Note that, make sure the elements in the arr_list, legend_list and color_list are associated with each other correctly.
        Do not forget to change the ylabel for different plots.
    """
    # set the figure type
    fig, ax = plt.subplots(figsize=(12, 8))

    # PLEASE NOTE: Change the labels for different plots
    ax.set_ylabel(ylabel)
    ax.set_xlabel("Time Steps")

    # ploth results
    h_list = []
    for arr, legend, color in zip(arr_list, legend_list, color_list):
        # compute the standard error
        arr_err = arr.std(axis=0) / np.sqrt(arr.shape[0])
        # plot the mean
        h, = ax.plot(range(arr.shape[1]), arr.mean(axis=0), color=color, label=legend)
        # plot the confidence band
        arr_err *= 1.96
        ax.fill_between(range(arr.shape[1]), arr.mean(axis=0) - arr_err, arr.mean(axis=0) + arr_err, alpha=0.3,
                        color=color)
        # save the plot handle
        h_list.append(h)

    # plot legends
    ax.set_title(f"{fig_title}")
    ax.legend(handles=h_list)

    plt.show()


In [ ]:
if __name__ == '__main__':

    print("Initializing doom...")
    game = vzd.DoomGame()
    game.load_config(os.path.join(vzd.scenarios_path, "basic.cfg"))
    game.set_window_visible(False)
    game.set_mode(vzd.Mode.PLAYER)
    game.set_screen_format(vzd.ScreenFormat.GRAY8)
    game.set_screen_resolution(vzd.ScreenResolution.RES_640X480)
    game.init()
    print("Doom initialized.")

    n = game.get_available_buttons_size()
    actions = [list(a) for a in it.product([0, 1], repeat=n)]

    train_parameters = {
        'learning_rate': 0.00025,
        'discount_factor': 0.99,
        'total_time_steps': 500000,
        'replay_buffer_size': 10000,
        'batch_size': 64,
        'frame_repeat': 12,
        'resolution': (30, 45),
        'episodes_to_watch': 5,
        'device': torch.device('cuda'),
        'epsilon': 1,
        'epsilon_decay': 0.9996,
        'epsilon_min': 0.1,
        'actions': actions
    }

    agent, game, returns, loss = train_mc_agent(game, train_parameters)

    game.close()
    game.set_window_visible(True)
    game.set_mode(vzd.Mode.ASYNC_PLAYER)
    game.init()

    visualize(game, agent, train_parameters)
    game.close()


In [ ]:
plot_curves([np.array([returns])], ['monte carlo'], ['b'], 'episodes', 'Vizdoom Deep-Monte Carlo')

In [ ]:
game.set_window_visible(True)
game.set_mode(vzd.Mode.ASYNC_PLAYER)
game.init()

visualize(game, agent, train_parameters)
game.close()